# Cleaning Agency Names and Locodes

* Cleaning Obligated Data using Crosswalks created in data_docs.ipynb

In [1]:
import pandas as pd
from siuba import *

import numpy as np

from datetime import date
from IPython.display import Markdown, HTML, display_html

from calitp import *

In [2]:
df = pd.read_csv('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/clean_obligated_waiting.csv', low_memory=False).drop('Unnamed: 0', axis=1)

/opt/conda/lib/python3.9/site-packages/google/auth/_default.py:73: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


## Makings of the Function:

In [3]:
cw1_locode =  pd.read_csv('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/agencylocode_primary_crosswalk1.csv')

In [4]:
cw2_id = pd.read_csv('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/null_locodes_crosswalk2.csv')

In [5]:
cw3_unmatched = pd.read_csv('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/unmatched_estimate.csv')

In [6]:
df.head()

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,project_location,type_of_work,seq,date_request_initiated,date_completed_request,mpo,warning,projectID,projectNO,compare_id_locode
0,Obligated,BPMPL,5904(121),Humboldt County,2018-12-18,2018-12-18,2018-12-18,2018-12-18,2018-12-27,0.00,...,14 Bridges In Humboldt County,Bridge Preventive Maintenance - Deck Joints,3,NaN,NaN,NON-MPO,NaN,5904,121,True
1,Obligated,ER,32D0(008),Mendocino County,2018-12-17,2018-12-19,2018-12-20,2018-12-20,2018-12-27,11508.00,...,"Comptche Ukiah Road, Cr 223 Pm 17.25",Permanent Restoration,3,2018-12-17,2018-12-18,NON-MPO,NaN,32D0,8,False
2,Obligated,ER,4820(004),Humboldt County,2018-12-07,2018-12-21,2018-12-21,2018-12-21,2018-12-27,45499.64,...,Mattole Rd Pm 43.17,Permanent Restoration,5,2018-12-06,2018-12-07,NON-MPO,NaN,4820,4,False
3,Obligated,CML,5924(244),Sacramento County,2018-12-11,2018-12-11,2018-12-21,2018-12-27,2018-12-27,207002.00,...,Fair Oaks Blvd. Between Howe Ave And Munroe St,Create A Smart Growth Corridor With Barrier Se...,1,2018-12-07,2018-12-07,SACOG,NaN,5924,244,True
4,Obligated,CML,5924(214),Sacramento County,2018-12-05,2018-12-11,2018-12-21,2018-12-27,2018-12-27,0.00,...,Florin Rd Between Power Inn Rd. And Florin Per...,Streetscape (tc),3,2018-11-28,2018-12-04,SACOG,NaN,5924,214,True


In [6]:
def get_num(x):
    try:
        return int(x)
    except Exception:
        try:
            return float(x)
        except Exception:
            return x

In [12]:
df['locode'] = df['locode'].apply(get_num)
df['projectID']= df['projectID'].apply(get_num)

In [13]:
locode_map1 = dict(zip(cw1_locode['primary_agency_locode'], 
                          cw1_locode['primary_agency_name']))

In [14]:
df['primary_agency_name'] = df['locode'].map(locode_map1)

In [15]:
locode_map2 = dict(zip(cw2_id['primary_agency_locode'], 
                          cw2_id['primary_agency_name']))

In [16]:
df['primary_agency_name_2'] = df['projectID'].map(locode_map2)

In [17]:
df['primary_agency_name'] = np.where(df.primary_agency_name.isnull(), 
                      df['primary_agency_name_2'],df['primary_agency_name'])
#df['primary_agency_name'] = np.where(df['primary_agency_name'] == 0, df['primary_agency_name_2'], df['primary_agency_name'])

In [18]:
df.sample(10)

,location,prefix,project_no,agency,prepared_date,submit__to_hq_date,hq_review_date,submit_to_fhwa_date,to_fmis_date,fed_requested,...,seq,date_request_initiated,date_completed_request,mpo,warning,projectID,projectNO,compare_id_locode,primary_agency_name,primary_agency_name_2
3298,Obligated,HSIP,5385(051),Ridgecrest,2017-10-04,2017-10-04,2017-10-04,2017-10-04,2017-10-19,-189903.72,...,4,NaN,NaN,KCOG,NaN,5385,51,True,Ridgecrest,Ridgecrest
3967,Obligated,CML,5006(729),Los Angeles,2017-05-16,2017-07-01,2017-07-06,2017-07-06,2017-07-17,-283000.00,...,2,2017-05-09,2017-06-23,SCAG,NaN,5006,729,True,Los Angeles,Los Angeles
10735,Obligated,RPSTP,5015(020),Placerville,2014-09-13,2014-09-13,2014-09-13,2014-09-13,2014-09-16,95000.00,...,3,NaN,NaN,SACOG,NaN,5015,20,True,Placerville,NaN
5195,Obligated,CML,5472(019),Windsor,2016-12-07,2016-12-09,2016-12-15,2016-12-16,2016-12-23,432000.00,...,1,NaN,NaN,MTC,NaN,5472,19,True,Windsor,NaN
15431,Obligated,STP,5060(330),Fresno,2019-12-06,2019-12-06,2019-12-06,2019-12-06,2019-12-17,-226.78,...,3,2019-11-28,2019-11-29,CFCG,NaN,5060,330,True,Fresno,NaN
8377,Obligated,BRLO,5277(028),Fairfax,2015-06-25,2015-08-05,2015-08-24,2015-08-24,2015-08-24,15000.00,...,1,NaN,NaN,MTC,NaN,5277,28,True,Fairfax,NaN
12246,Obligated,STP,5301(010),Orange Cove,2014-04-04,2014-04-04,2014-04-04,2014-04-04,2014-04-06,0.00,...,4,NaN,NaN,CFCG,NaN,5301,10,True,Orange Cove,NaN
10078,Obligated,ER,18D3(031),Highland,2015-01-12,2015-01-12,2015-01-12,2015-01-12,2015-01-14,-63.43,...,2,NaN,NaN,SCAG,NaN,18D3,31,False,Highland,NaN
15810,Obligated,HSIP,5066(045),National City,2020-01-14,2020-01-14,2020-01-15,2020-01-15,2020-01-27,175200.00,...,1,2019-12-19,2020-01-10,SDAG,NaN,5066,45,True,National City,National City
11983,Obligated,BRLO,5942(234),Fresno County,2014-02-26,2014-04-24,2014-04-28,2014-04-28,2014-04-29,579000.00,...,1,NaN,NaN,CFCG,NaN,5942,234,True,Fresno County,NaN


In [19]:
len(df>>filter(_.primary_agency_name.isnull())>>filter(_.primary_agency_name_2.isnull()))

57

In [20]:
(df
 >>filter(_.primary_agency_name.isnull())
 >>filter(_.primary_agency_name_2.isnull())
 >>count(_.agency)
 >> arrange(-_.n)
)

,agency,n
6,Inyo,14
9,Lancaster,6
25,Yucaipa,3
1,Banning,2
3,Fowler,2
4,Gold Coast Transit District,2
8,La Quinta,2
10,Los Angeles,2
12,Palmdale,2
14,San Bernardino,2


In [24]:
#cw3_unmatched
## not running beacause of an error

In [25]:
# locode_map3 = dict(zip(cw3_unmatched['agency_locode'], 
#                           cw3_unmatched['primary_agency_name']))

In [26]:
# df['primary_agency_name_3'] = df['projectID'].map(locode_map3)
# df['primary_agency_name_4'] = df['locode'].map(locode_map3)

## Converting to a Function

In [52]:
#25 agencies in the unmatched df. 

In [54]:
#may need to change `agency_locode_id` column name to make this work 

In [10]:
def clean_agency_names(df):
    
    df =  pd.read_csv('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/clean_obligated_waiting.csv', low_memory=False).drop('Unnamed: 0', axis=1)
    
    cw1_locode =  pd.read_csv('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/agencylocode_primary_crosswalk1.csv')
    cw2_id = pd.read_csv('gs://calitp-analytics-data/data-analyses/dla/e-76Obligated/null_locodes_crosswalk2.csv')
    
    # ensure the values in crosswalk are int64 
    
    # df['locode'] = df['locode'].apply(get_num)
    # df['projectID']= df['projectID'].apply(get_num)
    
    # create a dictionary for the data
    
    locode_map1 = dict(zip(cw1_locode['primary_agency_locode'], 
                          cw1_locode['primary_agency_name']))
    
    locode_map2 = dict(zip(cw2_id['primary_agency_locode'], 
                          cw2_id['primary_agency_name']))
    
    df['primary_agency_name'] = df['locode'].map(locode_map1)
    df['primary_agency_name2'] = df['projectID'].map(locode_map2)
    
    return df

In [11]:
df_clean = clean_agency_names(df)